In [9]:
import code_ablation  
import code

In [10]:
text = """If the rabbit is round and the rabbit needs the squirrel then the rabbit is rough.If something needs the cat and the cat chases the rabbit then the cat chases the bear.If something is big then it chases the rabbit.If something sees the cat and the cat is blue then the cat visits the dog.If something is cold then it visits the bear.If something is green and it visits the bear then the bear sees the dog.If Bob is not furry then Bob is round.If Erin is round then Erin is white.If someone is furry and not rough then they are blue.If someone is furry and not round then they are blue.If someone is round and not red then they are nice.If someone is white and not blue then they are quiet.Anne is blue. Anne is furry. Anne is nice. Anne is red. Anne is smart. Bob is kind. Bob is nice. If something is furry then it is kind. Nice, smart things are kind. All nice things are furry. If Bob is smart and Bob is blue then Bob is kind. If something is kind then it is nice. Kind things are smart.All rough things are smart. All white, rough things are smart. All nice things are red. All white things are nice. Kind things are white. All kind, round things are rough. All nice, young things are round. Nice, young things are round. Green things are round. Round, green things are rough. Nice, young things are green. All rough, green things are young. All round things are nice. If the lion is round and the lion is young then the lion is rough."""


facts_rules_list = code.FR_decomposer_proofwriter_new(text)
facts = facts_rules_list['facts']
rules = facts_rules_list['rules']

In [11]:
rules

['If the rabbit is round and the rabbit needs the squirrel then the rabbit is rough.',
 'If something needs the cat and the cat chases the rabbit then the cat chases the bear.',
 'If something is big then it chases the rabbit.',
 'If something sees the cat and the cat is blue then the cat visits the dog.',
 'If something is cold then it visits the bear.',
 'If something is green and it visits the bear then the bear sees the dog.',
 'If Bob is not furry then Bob is round.',
 'If Erin is round then Erin is white.',
 'If someone is furry and not rough then they are blue.',
 'If someone is furry and not round then they are blue.',
 'If someone is round and not red then they are nice.',
 'If someone is white and not blue then they are quiet.',
 'If something is furry then it is kind.',
 'Nice, smart things are kind.',
 'All nice things are furry.',
 'If Bob is smart and Bob is blue then Bob is kind.',
 'If something is kind then it is nice.',
 'Kind things are smart.',
 'All rough things ar

In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
from transformers import AutoTokenizer
import transformers
import torch
import json
import pandas as pd
from tqdm import tqdm
import logging
import time

model = "meta-llama/Llama-2-7b-chat-hf"
# model = "meta-llama/Llama-2-13b-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)
def ask(question:str, max_length=500) -> str:
    sequences = pipeline(
        question,
        do_sample=False,
        # do_sample=True,
        # top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=max_length,
        # max_length=200,
    )
    for seq in sequences:
        return seq['generated_text']


/home/yanghn/env_py/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:43<00:00, 21.75s/it]


In [14]:
facts_rules_list['rules']

['If the rabbit is round and the rabbit needs the squirrel then the rabbit is rough.',
 'If something needs the cat and the cat chases the rabbit then the cat chases the bear.',
 'If something is big then it chases the rabbit.',
 'If something sees the cat and the cat is blue then the cat visits the dog.',
 'If something is cold then it visits the bear.',
 'If something is green and it visits the bear then the bear sees the dog.',
 'If Bob is not furry then Bob is round.',
 'If Erin is round then Erin is white.',
 'If someone is furry and not rough then they are blue.',
 'If someone is furry and not round then they are blue.',
 'If someone is round and not red then they are nice.',
 'If someone is white and not blue then they are quiet.',
 'If something is furry then it is kind.',
 'Nice, smart things are kind.',
 'All nice things are furry.',
 'If Bob is smart and Bob is blue then Bob is kind.',
 'If something is kind then it is nice.',
 'Kind things are smart.',
 'All rough things ar

### Rules

In [136]:
rules = ['if jack loves rose then jack is blue',
    'if bob is blue, then bob is green',
    'all nice cat are cute',
    'if bob is nice, then bob is dooog',
    'if something is cute, then it is nice',
    'if something is not cute, then it is real',
    'if cat is cute and loves dog, then it likes lion'
    'all nice, handsome cats are cute',
    'all nice, round, rough cats are cute',
    'red, blue cats are green',
    ]

In [138]:

def prompt_formulator_translation_rules_2_ifthen_4_example(rules:list) -> str:
    prompt = f"""Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob is good, then Bob is nice, output: IF Good(Bob) THEN Nice(Bob)
Example: Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Example: Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
Example: Sentence: All cute dog are nice, output: IF Cute(dog) THEN Nice(dog)
Example: Sentence: Good people are nice, output: IF Good(people) THEN Nice(people)
Sentences: 
{code.list_2_str(rules)}Output:
"""
    return prompt

prompt_rule = prompt_formulator_translation_rules_2_ifthen_4_example(rules)
answer = ask(prompt_rule, max_length=700)
print(answer)

/home/yanghn/env_py/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob is good, then Bob is nice, output: IF Good(Bob) THEN Nice(Bob)
Example: Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Example: Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
Example: Sentence: All cute dog are nice, output: IF Cute(dog) THEN Nice(dog)
Example: Sentence: Good people are nice, output: IF Good(people) THEN Nice(people)
Sentences: 
*if jack loves rose then jack is blue
*if bob is blue, then bob is green
*all nice cat are c

In [141]:

def prompt_formulator_translation_rules_2_ifthen_4_example(rules:list) -> str:
    prompt = f"""Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob is good, then Bob is nice, output: IF Good(Bob) THEN Nice(Bob)
Example: Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Example: Sentence: All cute dog are nice, output: IF Cute(dog) THEN Nice(dog)
Example: Sentence: All cute, rough cats are nice, output: IF Cute(cat) AND Rough(cat) THEN Nice(dog)
Example: Sentence: Good people are nice, output: IF Good(people) THEN Nice(people)
Sentences: 
{code.list_2_str(rules)}Output:
"""
    return prompt

prompt_rule = prompt_formulator_translation_rules_2_ifthen_4_example(rules)
answer = ask(prompt_rule, max_length=700)
print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob is good, then Bob is nice, output: IF Good(Bob) THEN Nice(Bob)
Example: Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Example: Sentence: All cute dog are nice, output: IF Cute(dog) THEN Nice(dog)
Example: Sentence: All cute, rough cats are nice, output: IF Cute(cat) AND Rough(cat) THEN Nice(dog)
Example: Sentence: Good people are nice, output: IF Good(people) THEN Nice(people)
Sentences: 
*if jack loves rose then jack is blue
*if bob is blue, then bob is green
*all 

In [143]:

def prompt_formulator_translation_rules_2_ifthen_4_example(rules:list) -> str:
    prompt = f"""Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Example: Sentence: All cute dog are nice, output: IF Cute(dog) THEN Nice(dog)
Example: Sentence: Good people are nice, output: IF Good(people) THEN Nice(people)
Sentences: 
{code.list_2_str(rules)}Output:
"""
    return prompt

prompt_rule = prompt_formulator_translation_rules_2_ifthen_4_example(rules)
answer = ask(prompt_rule, max_length=700)
print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Example: Sentence: All cute dog are nice, output: IF Cute(dog) THEN Nice(dog)
Example: Sentence: Good people are nice, output: IF Good(people) THEN Nice(people)
Sentences: 
*if jack loves rose then jack is blue
*if bob is blue, then bob is green
*all nice cat are cute
*if bob is nice, then bob is dooog
*if something is cute, then it is nice
*if something is not cute, then it is real
*if cat is cute and loves dog, then it likes lionall ni

In [144]:

def prompt_formulator_translation_rules_2_ifthen_4_example(rules:list) -> str:
    prompt = f"""Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Example: Sentence: All cute dog are nice, output: IF Cute(dog) THEN Nice(dog)
Example: Sentence: Good people are nice, output: IF Good(people) THEN Nice(people)
Sentences: 
{code.list_2_str(rules)}Output:
"""
    return prompt

prompt_rule = prompt_formulator_translation_rules_2_ifthen_4_example(rules)
answer = ask(prompt_rule, max_length=700)
print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Example: Sentence: All cute dog are nice, output: IF Cute(dog) THEN Nice(dog)
Example: Sentence: Good people are nice, output: IF Good(people) THEN Nice(people)
Sentences: 
*if jack loves rose then jack is blue
*if bob is blue, then bob is green
*all nice cat are cute
*if bob is nice, then bob is dooog
*if something is cute, then it is nice
*if something is not cute, then it is real
*if cat is cute and loves dog, then it likes lionall nice, handsome cats are cute
*all nice, round, rough cats are cute
*red, blue cats are green
Output:
*IF Love(Jack

In [145]:

def prompt_formulator_translation_rules_2_ifthen_4_example(rules:list) -> str:
    prompt = f"""Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Example: Sentence: All cute dog are nice, output: IF Cute(dog) THEN Nice(dog)
Sentences: 
{code.list_2_str(rules)}Output:
"""
    return prompt

prompt_rule = prompt_formulator_translation_rules_2_ifthen_4_example(rules)
answer = ask(prompt_rule, max_length=700)
print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Example: Sentence: All cute dog are nice, output: IF Cute(dog) THEN Nice(dog)
Sentences: 
*if jack loves rose then jack is blue
*if bob is blue, then bob is green
*all nice cat are cute
*if bob is nice, then bob is dooog
*if something is cute, then it is nice
*if something is not cute, then it is real
*if cat is cute and loves dog, then it likes lionall nice, handsome cats are cute
*all nice, round, rough cats are cute
*red, blue cats are green
Output:
*IF Love(Jack, Rose) THEN Blue(Jack)
*IF Blue(Bob) THEN Green(Bob)
*ALL Nice(Cat) THEN Cute(Cat)

In [146]:
def prompt_formulator_translation_rules_2_ifthen_4_example(rules:list) -> str:
    prompt = f"""Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Sentences: 
{code.list_2_str(rules)}Output:
"""
    return prompt

prompt_rule = prompt_formulator_translation_rules_2_ifthen_4_example(rules)
answer = ask(prompt_rule, max_length=700)
print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Example: Sentence: If Bob likes dog, then Bob is nice, output: IF Like(Bob, dog) THEN Nice(Bob)
Sentences: 
*if jack loves rose then jack is blue
*if bob is blue, then bob is green
*all nice cat are cute
*if bob is nice, then bob is dooog
*if something is cute, then it is nice
*if something is not cute, then it is real
*if cat is cute and loves dog, then it likes lionall nice, handsome cats are cute
*all nice, round, rough cats are cute
*red, blue cats are green
Output:
*IF Love(Jack, Rose) THEN Blue(Jack)
*IF Blue(Bob) THEN Green(Bob)
*ALL Nice(Cat) THEN Cute(Cat)
*IF Nice(Bob) THEN Dooog(Bob)
*IF Cute(X) THEN Nice(X)
*IF Not Cute(X) THEN R

In [149]:
def prompt_formulator_translation_rules_2_ifthen_4_example(rules:list) -> str:
    prompt = f"""Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Sentences: 
{code.list_2_str(rules)}Output:
"""
    return prompt

prompt_rule = prompt_formulator_translation_rules_2_ifthen_4_example(rules)
answer = ask(prompt_rule, max_length=700)
print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property> <Object> are <Property>" should be written as "IF <Property>(<Object>), THEN <Property>(<Object>)".
Sentences: 
*if jack loves rose then jack is blue
*if bob is blue, then bob is green
*all nice cat are cute
*if bob is nice, then bob is dooog
*if something is cute, then it is nice
*if something is not cute, then it is real
*if cat is cute and loves dog, then it likes lionall nice, handsome cats are cute
*all nice, round, rough cats are cute
*red, blue cats are green
Output:
Please convert the following rules into IF-THEN expressions:

1. IF Jack loves Rose, THEN Jack is blue.
2. IF Bob is blue, THEN Bob is green.
3. ALL nice cat ARE cute.
4. IF Bob is nice, THEN Bob is dooog.
5. IF something is cute, THEN it is nice.
6. IF something is not

# Evaluate Rules

In [15]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

In [119]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

def rules_parse_answer(answer:str) -> dict:
    # convert answer to {condition:{set}, conclusion:{set}}
    if 'if' not in answer.lower() or 'then' not in answer.lower():
        return {answer:answer}
    else: 
        words = answer.lower().split('then')    # ['if xxx'] then ['xxx']
        if len(words) < 2:
            return {answer:answer}
        return find_answer_rules(words[0], words[1])

def rules_parse_rule(rule:str) -> dict:
    # convert rule sentence to {condition:{set}, conclusion:{set}}
    # can't find solution for 'are', or 'all'
    rule_lower = rule.lower()
    rule_lower = rule_lower.replace('all', '')
    rule_lower = rule_lower.replace('if', '')
    rule_lower = rule_lower.replace('is', '')
    rule_lower = rule_lower.replace('.', '')
    rule_lower = rule_lower.replace(',', '')
    if 'are' in rule_lower:
        # words = rule_lower.split('are')
        # if len(words) < 2:
        return {rule:rule}
    elif 'then' in rule_lower:
        words = rule_lower.split('then')
        if len(words) < 2:
            return {rule:rule}
    else: 
        return {rule:rule}
    return find_answer_rules(words[0], words[1])


def find_answer_rules(str_conditions:str, str_conclusion:str) -> dict:
    lemmatizer = WordNetLemmatizer()
    result = {
        'conditions'    : [],
        'conclusion'    : [],
    }
    str_conditions = str_conditions.strip()
    str_conclusion = str_conclusion.strip()
    

    # process condition
    str_conditions = str_conditions.replace('(', ' ')
    str_conditions = str_conditions.replace(')', ' ')
    str_conditions = str_conditions.replace('if', '')
    str_conditions = str_conditions.replace('and', '')
    str_conditions = str_conditions.replace('is', '')
    str_conditions = str_conditions.replace(',', '')

    str_conditions = str_conditions.strip()

    tokens = word_tokenize(str_conditions)
    tagged_words = pos_tag(tokens)
    nouns = [word[0] for word in tagged_words if word[1].startswith('NN')]
    nouns = list(set(nouns))
    adjs  = [word[0] for word in tagged_words if word[1].startswith('JJ')]
    adjs  = list(set(adjs))
    verbs = [word[0] for word in tagged_words if word[1].startswith('VB') and word[0] not in ['is', 'are']]
    verbs = list(set(verbs))
    nouns = [lemmatizer.lemmatize(word, pos='n') for word in nouns]
    adjs  = [lemmatizer.lemmatize(word, pos='a') for word in adjs ]
    verbs = [lemmatizer.lemmatize(word, pos='v') for word in verbs]
    result['conditions'].extend(nouns)
    result['conditions'].extend(adjs)
    result['conditions'].extend(verbs)

    # process conclusion
    str_conclusion = str_conclusion.replace('(', ' ')
    str_conclusion = str_conclusion.replace(')', ' ')
    str_conclusion = str_conclusion.replace('if', '')
    str_conclusion = str_conclusion.replace('and', '')
    str_conclusion = str_conclusion.replace('is', '')
    str_conclusion = str_conclusion.replace('are', '')
    str_conclusion = str_conclusion.replace(',', '')
    str_conclusion = str_conclusion.strip()

    tokens = word_tokenize(str_conclusion)
    tagged_words = pos_tag(tokens)
    nouns = [word[0] for word in tagged_words if word[1].startswith('NN')]
    nouns = list(set(nouns))
    adjs  = [word[0] for word in tagged_words if word[1].startswith('JJ')]
    adjs  = list(set(adjs))
    verbs = [word[0] for word in tagged_words if word[1].startswith('VB') and word[0] not in ['is', 'are']]
    verbs = list(set(verbs))
    nouns = [lemmatizer.lemmatize(word, pos='n') for word in nouns]
    adjs  = [lemmatizer.lemmatize(word, pos='a') for word in adjs ]
    verbs = [lemmatizer.lemmatize(word, pos='v') for word in verbs]
    result['conclusion'].extend(nouns)
    result['conclusion'].extend(adjs)
    result['conclusion'].extend(verbs)

    result['conditions'] = set(result['conditions'])
    result['conclusion'] = set(result['conclusion'])
    print(result)
    return result

def evaluation_translation_rule(rules_list:list, answers_list:list):
    # how many examples in answer?
    # how many are correct?
    answer_parsed = [rules_parse_answer(answer) for answer in answers_list]
    
    num_correct = 0
    rules_no_answer = []
    rules_are = [rule for rule in rules_list if 'are' in rule]
    num_rules_are = len(rules_are)
    for rule in rules_list:
        if 'are' in rule:
            continue
        rule_parsed = rules_parse_rule(rule)
        if rule_parsed in answer_parsed:
            num_correct += 1
            index = answer_parsed.index(rule_parsed)
            answers_list.remove(answers_list[index])
            answer_parsed.remove(rule_parsed)
        else:
            rules_no_answer.append(rule)

    bad_answers = answers_list
    num_rules_no_answer = len(rules_no_answer)
    num_bad_answers = len(bad_answers)

    str_rules_no_answer = ""
    for index, rule in enumerate(rules_no_answer):
        str_rules_no_answer += f"{index+1}. {rule}\n"

    str_bad_answers     = ""
    for index, rule in enumerate(bad_answers):
        str_bad_answers += f"{index+1}. {rule}\n"

    str_are_rules     = ""
    for index, rule in enumerate(rules_are):
        str_are_rules += f"{index+1}. {rule}\n"

    result_dict = {}
    result_dict['num_all']                       = len(rules_list)
    result_dict['num_rules_without_are']         = len(rules_list) - num_rules_are
    result_dict['num_correct']                   = num_correct
    result_dict['num_rules_no_answer']           = num_rules_no_answer
    result_dict['num_bad_answers']               = num_bad_answers
    result_dict['str_rules_no_answer']           = str_rules_no_answer
    result_dict['str_bad_answers']               = str_bad_answers
    result_dict['num_rules_are']                 = num_rules_are
    result_dict['str_are_rules']                 = str_are_rules
    return result_dict


In [18]:
'if you are good'.split('1')

['if you are good']

In [17]:
a = 'and'
a.replace('and', ",")
a

'and'

In [82]:
5 in {1:1}

False

In [19]:
tokens = word_tokenize('If something needs the cat and the cat chases the rabbit then the cat chases the bear.')
# split with if, and, then
tagged_words = pos_tag(tokens)
tagged_words

[('If', 'IN'),
 ('something', 'NN'),
 ('needs', 'VBZ'),
 ('the', 'DT'),
 ('cat', 'NN'),
 ('and', 'CC'),
 ('the', 'DT'),
 ('cat', 'NN'),
 ('chases', 'VBZ'),
 ('the', 'DT'),
 ('rabbit', 'NN'),
 ('then', 'RB'),
 ('the', 'DT'),
 ('cat', 'NN'),
 ('chases', 'VBZ'),
 ('the', 'DT'),
 ('bear', 'NN'),
 ('.', '.')]

In [24]:
nouns = [word[0] for word in tagged_words if word[1].startswith('NN')]
list(set(nouns))

['things', 'dog', 'cat']

In [26]:
adj = [word[0] for word in tagged_words if word[1].startswith('JJ')]
list(set(adj))

['blue']

In [30]:
verb = [word[0] for word in tagged_words if word[1].startswith('VB') and word[0] not in ['is', 'are']]
list(set(verb))

['sees', 'visits']

In [32]:
a = "asd"
a.replace(a, "")
a

'asd'

In [34]:
tokens = word_tokenize('nice(bob)')# split with if, and, then
tagged_words = pos_tag(tokens)
tagged_words

[('nice', 'JJ'), ('(', '('), ('bob', 'NN'), (')', ')')]

In [58]:
parse_answer_rules(text)

{'conditions': {'need', 'rabbit', 'round', 'squirrel'},
 'conclusion': {'cat', 'chase', 'need', 'rabbit', 'rough', 'something'}}

In [42]:
['a', 'b'] == ['a', 'c']

False

In [43]:
['a', 'b'] == ['a', 'b']

True

In [44]:
['a', 'b'] == ['b', 'a']

False

In [45]:
{'a', 'b'} == {'a', 'b'}

True

In [46]:
{'a', 'b'} == {'b', 'a'}

True

In [48]:
print(type({'b', 'a'}))

<class 'set'>


In [56]:
{'b':{}, 'a':{'a', 'b'}} == {'a':{'b', 'a'},'b':{}} 

True

In [59]:
parse_answer_rules(text) == parse_answer_rules(text)

True

In [63]:
parse_answer_rules('if bob like jack and bob is nice then bob is blue')


{'conditions': {'bob', 'jack', 'nice'}, 'conclusion': {'blue', 'bob'}}

In [65]:
parse_answer_rules('IF like(bob, jack) AND nice(bob) THEN Blue(bob) ')

{'conditions': {'bob', 'jack', 'nice'}, 'conclusion': {'blue', 'bob'}}

In [73]:
a = [(1,2), (2,3), 2]
a.remove((2,3))
a

[(1, 2), 2]

In [74]:
type({})

dict

In [80]:
l = [1,2,3]
list({i:str(i) for i in l}.keys())

[1, 2, 3]

In [111]:
rules_parse_rule('nice bob are cute',)

{'nice bob are cute': 'nice bob are cute'}

In [112]:
rules_parse_rule('all nice bob are cute',)

{'all nice bob are cute': 'all nice bob are cute'}

In [106]:
print(rules_parse_answer('if nice(bob) then cute(bob)'))

{'conditions': {'nice', 'bob'}, 'conclusion': {'cute', 'bob'}}
{'conditions': {'nice', 'bob'}, 'conclusion': {'cute', 'bob'}}


In [103]:
find_answer_rules('if nice(bob)', 'cute(bob)')

{'conditions': {'nice', 'bob'}, 'conclusion': {'cute', 'bob'}}


{'conditions': {'bob', 'nice'}, 'conclusion': {'bob', 'cute'}}

In [121]:
evaluation_translation_rule([
    'if jack loves rose then jack is blue',
    'if bob is blue, then bob is green',
    'all nice bob are cute',
    'if bob is nice, then bob is dooog'
],
    [
        'IF love(jack, rose) THEN blue(jack)',  # 'if jack loves rose then jack is blue'
        'if green(bob) then nice(bob)',
        'if nice(bob) then cute(bob)',          # 'all nice bob are cute',
        # 'asd',
        'if cute(bob) then good(bob)',
        'if nice(bob) then dooog(bob)',
        
    ])

{'conditions': {'rise', 'jack', 'love'}, 'conclusion': {'jack', 'blue'}}
{'conditions': {'green', 'bob'}, 'conclusion': {'nice', 'bob'}}
{'conditions': {'nice', 'bob'}, 'conclusion': {'cute', 'bob'}}
{'conditions': {'cute', 'bob'}, 'conclusion': {'good', 'bob'}}
{'conditions': {'nice', 'bob'}, 'conclusion': {'dooog', 'bob'}}
{'conditions': {'rise', 'jack', 'love'}, 'conclusion': {'jack', 'blue'}}
{'conditions': {'bob', 'blue'}, 'conclusion': {'green', 'bob'}}
{'conditions': {'nice', 'bob'}, 'conclusion': {'dooog', 'bob'}}


{'num_all': 4,
 'num_rules_without_are': 3,
 'num_correct': 2,
 'num_rules_no_answer': 1,
 'num_bad_answers': 3,
 'str_rules_no_answer': '0. if bob is blue, then bob is green\n',
 'str_bad_answers': '0. if green(bob) then nice(bob)\n1. if nice(bob) then cute(bob)\n2. if cute(bob) then good(bob)\n',
 'num_rules_are': 1,
 'str_are_rules': '0. all nice bob are cute\n'}

In [87]:
{
    list({1:1}) : 5
}

TypeError: unhashable type: 'list'

In [89]:
a = [1,1]
a.remove(1)
a

[1]

In [90]:
a = [1,2]
a.index(1)


0

In [92]:
'aaa'.find('a')

0